In [1]:
import os

In [2]:
%pwd

'e:\\PWSkills\\MLFlow-End-to-end-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\PWSkills\\MLFlow-End-to-end-project'

In [5]:
import pandas as pd

In [6]:
df=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1591,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1592,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1593,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1594,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1596 non-null   float64
 1   volatile acidity      1596 non-null   float64
 2   citric acid           1596 non-null   float64
 3   residual sugar        1596 non-null   float64
 4   chlorides             1596 non-null   float64
 5   free sulfur dioxide   1596 non-null   float64
 6   total sulfur dioxide  1596 non-null   float64
 7   density               1596 non-null   float64
 8   pH                    1596 non-null   float64
 9   sulphates             1596 non-null   float64
 10  alcohol               1596 non-null   float64
 11  quality               1596 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 149.8 KB


In [8]:
print(df.columns)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


In [9]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [10]:
df.shape

(1596, 12)

In [11]:
if df['density'].dtype == 'float64':
    print("True")

True


In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict
    

In [16]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config =read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config

In [25]:
import os
from mlflow_project import logger

class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            df=pd.read_csv(self.config.unzip_data_dir)
            all_columns=list(df.columns)
            all_schema=self.config.all_schema
            

            for column in all_columns:
                if column not in all_schema.keys() or df[column].dtype != all_schema[column] :
                    logger.info(f"validating status as false for column:{column}")
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status : {validation_status}")

                else:
                    validation_status=True
                    logger.info(f"validating status as true for column:{column}")
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status : {validation_status}")

        except Exception as e:
            raise e
                    



In [28]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise e


[2023-11-06 00:25:28,941: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-06 00:25:28,952: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-06 00:25:28,958: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-06 00:25:28,966: INFO: common: created directory at: artifacts]
[2023-11-06 00:25:28,966: INFO: common: created directory at: artifacts/data_validation]
[2023-11-06 00:25:28,982: INFO: 1543085674: validating status as true for column:fixed acidity]
[2023-11-06 00:25:28,984: INFO: 1543085674: validating status as true for column:volatile acidity]
[2023-11-06 00:25:28,989: INFO: 1543085674: validating status as true for column:citric acid]
[2023-11-06 00:25:28,989: INFO: 1543085674: validating status as true for column:residual sugar]
[2023-11-06 00:25:28,997: INFO: 1543085674: validating status as true for column:chlorides]
[2023-11-06 00:25:28,997: INFO: 1543085674: validating status as true for column:free sulfur dioxide]